# Exercises Evaluation Metrics

In [223]:
import pandas as pd                                                     # Data manipulation
import numpy as np                                                      # Scientific computing
import matplotlib.pyplot as plt                                         # Data visualization
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis    # Discriminant Analysis

def trueFalse (confusion_matrix, columnnb=0):
    TP = confusion_matrix.iloc[columnnb][columnnb]
    print('TP', TP)
    TN = np.diag(confusion_matrix).sum() - TP
    print('TN:', TN)
    FP = confusion_matrix.iloc[:, columnnb].sum() - TP
    print('FP:', FP)
    FN = confusion_matrix.iloc[columnnb,:].sum() - TP
    print('FN:', FN)
    return

def accuracy(confusion_matrix):
    return np.diag(confusion_matrix).sum()/confusion_matrix.sum().sum()

def precision(confusion_matrix):
    precision = []
    n = confusion_matrix.shape[1]
    for i in range(0,n):
        TP = confusion_matrix.iloc[i][i]
        precision = precision + [TP/confusion_matrix.iloc[:, i].sum()]
    return precision

def recall(confusion_matrix):
    recall = []
    n = confusion_matrix.shape[0]
    for i in range(0,n):
        TP = confusion_matrix.iloc[i][i]
        recall = recall + [TP/confusion_matrix.iloc[i, :].sum()]
    return recall

def f_measure(confusion_matrix, beta):
    precisionarray = precision(confusion_matrix)
    recallarray = recall(confusion_matrix)
    fmeasure=[]
    n = len(precisionarray)
    for i in range(0,n):
        p = precisionarray[i]
        r = recallarray [i]
        fmeasure = fmeasure + [((beta*beta+1)*p*r)/(beta*beta*p+r)]
    return fmeasure

def overview_metrieken(confusion_matrix, beta):
    overview_1 = np.transpose(precision (confusion_matrix))
    overview_2 = np.transpose(recall(confusion_matrix))
    overview_3 = np.transpose(f_measure(confusion_matrix,beta))
    overview_table=pd.DataFrame (data=np.array([overview_1, overview_2, overview_3]), columns=confusion_matrix.index)
    overview_table.index = ['precision', 'recall', 'fx']
    return[overview_table]

def positiverates(confusion_matrix):
    if (confusion_matrix.shape[0] == 2) & (confusion_matrix.shape[1] == 2):
        TPR = confusion_matrix.iloc[0][0]/confusion_matrix.iloc[0, :].sum()
        print('TPR', TPR)
        FPR = confusion_matrix.iloc[1][0]/confusion_matrix.iloc[1, :].sum()
        print('FPR', FPR)
    return

def plot_roc(y_true, y_score, title='ROC Curve', **kwargs):
    from sklearn.metrics import roc_curve, roc_auc_score
    
    if 'pos_label' in kwargs:
        fpr, tpr, thresholds = roc_curve(y_true=y_true, y_score=y_score, pos_label=kwargs.get('pos_label'))
        auc = roc_auc_score(y_true, y_score)
    else:
        fpr, tpr, thresholds = roc_curve(y_true=y_true, y_score=y_score)
        auc = roc_auc_score(y_true, y_score)

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    figsize = kwargs.get('figsize', (7, 7))
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.grid(linestyle='--')

    ax.plot(fpr, tpr, color='darkorange', label='AUC: {}'.format(auc))
    ax.set_title(title)
    ax.set_xlabel('False Positive Rate (FPR)')
    ax.set_ylabel('True Positive Rate (TPR)')
    ax.fill_between(fpr, tpr, alpha=0.3, color='darkorange', edgecolor='black')

    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    ax.scatter(fpr[optimal_idx], tpr[optimal_idx], label='optimal cutoff {:.2f} on ({:.2f},{:.2f})'.format(optimal_threshold, fpr[optimal_idx], tpr[optimal_idx]), color='red')
    ax.plot([fpr[optimal_idx], fpr[optimal_idx]], [0, tpr[optimal_idx]], linestyle='--', color='red')
    ax.plot([0, fpr[optimal_idx]], [tpr[optimal_idx], tpr[optimal_idx]], linestyle='--', color='red')

    ax.legend(loc='lower right')
    plt.show()

## Theoretical questions

### Question 1:
- We willen gaan voorspellen of studenten heel gemotiveerd zijn om een diploma Toegepaste Informatica te behalen.
    - De onafhankelijke gegevens (de voorspellers) zijn "het aantal broers en/of zussen", "aantal samenzweringen" en "het aantal uren wiskunde". Plaats de onafhankelijke variabelen in een nieuw dataframe met naam df en zorg ervoor dat de waarden van de voorspellers numeriek zijn en dat eventueel onrealistische gegevens worden omgezet in nan-waarden.
    - In de vragenlijst wil je het "belang van informatica" gaan voorspellen. Dit is een ordinaal gegeven dat je zal moeten reduceren tot twee mogelijke waarden: alles gelijk of meer dan "groot" is "groot", alles kleiner is "matig". Voeg het resultaat als een kolom toe aan het dataframe df.
    - Gebruik lineaire discriminant analyse als model om de afhankelijke variabele "belang van Informatica" te voorspellen met behulp van de onafhankelijke variabelen (voorspellers). Maak de voorspellingen voor de beschikbare gegevens.
    - Bestudeer de resultaten van de evaluatie metrieken:
        - Maak de confusion matrix
        - Bereken de accuracy, precision, recall en F1
        - Maak de ROC-curve

In [224]:
data1 = pd.read_csv('../Data/Data1.csv', delimiter=';', decimal=',')
display(data1.head())

,gapminder,schrijfhand,schoenmaat,lengte,bloedgroep,resus,siblings,Samenzweringen,rijbewijs,domicilie in Antwerpen,...,M1 - stelen,M2 - team,M3 - beschermen,M4 - voorbeeld,M5 - embryo,M6 - vrijheid,mascotte,zwaarste vak,boeienste vak,studiepunt
0,7/18,Rechts,43.0,184,O,positief,5,De tsunami van 2004 in de Indische Oceaan was ...,Categorie B: Wagens.,"buiten de stad (een dorp, een gehucht, het pla...",...,7,2,8,6,5,4.0,"1=Bever,2=Wasbeer,3=Otter,4=Wolf,5=Poema,6=Pan...",Management 1,Data science 1,16-20 uren
1,3/18,Rechts,42.0,163,Weet ik niet,Weet ik niet,1,Prinses Diana kwam niet per ongeluk om het leven,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,10,5,10,6,7,3.0,"1=Bever,2=Wolf,3=Bizon,4=Otter,5=Panter,6=Poem...",Management 1,Computersystemen 1,<= 10 uren
2,12/18,Rechts,45.0,183,AB,positief,2,Prinses Diana kwam niet per ongeluk om het lev...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,6,3,7,7,10,6.0,"1=Bruine beer,2=Poema,3=Panter,4=Wolf,5=Vos,6=...",Management 1,Programmeren 1 - Java,16-20 uren
3,5/18,Rechts,44.0,175,A,positief,2,De wereld wordt eigenlijk gecontroleerd door e...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,9,10,10,10,10,0.0,"1=Wasbeer,2=Wolf,3=Bruine beer,4=Das,5=Vos,6=B...",Management 1,Data science 1,<= 10 uren
4,6/18,Rechts,43.0,173,B,positief,3,De wereld wordt eigenlijk gecontroleerd door e...,Categorie B: Wagens.,in een stad,...,10,6,10,7,7,2.0,"1=Wolf,2=Vos,3=Bruine beer,4=Bizon,5=Panter,6=...",Management 1,Data science 1,<= 10 uren


### Question 2:
- In de tabel hieronder vind je de confusion matrix van een binaire classifier.

|            | Predicted YES | Predicted NO | Total |
|------------|---------------|--------------|-------|
| Actual YES | 100           | 5            | 105   |
| Actual NO  | 10            | 50           | 60    |
| Total      | 110           | 55           | 165   |

- Beantwoord nu de volgende vragen:
    - Wat zijn de waarden voor TP, TN, FP, FN?
    - Bereken nu de accuracy, precision en recall.
    - Bereken de F1 en F1.5-measures.
    - Wat is de TPR en FPR? Vergelijk met je recall. Wat stel je vast?

In [225]:
# 1
# YES = Positive
# NO = Negative
confusion_matrix1 = pd.DataFrame(data=[[100, 5], [10, 50]], columns=['Predicted YES', 'Predicted NO'], index=['Actual YES', 'Actual NO'])
display(confusion_matrix1)

,Predicted YES,Predicted NO
Actual YES,100,5
Actual NO,10,50


In [226]:
# 2
accuracy = np.diag(confusion_matrix1).sum()/confusion_matrix1.sum().sum()
print(f"Acuracy: {round(accuracy, 3)}")

precision_yes = precision(confusion_matrix1)[0]
print(f"Precision YES: {round(precision_yes, 3)}")
precision_no = precision(confusion_matrix1)[1]
print(f"Precision NO: {round(precision_no, 3)}")

recall_yes = recall(confusion_matrix1)[0]
print(f"Recall YES: {round(recall_yes, 3)}")
recall_no = recall(confusion_matrix1)[1]
print(f"Recall NO: {round(recall_no, 3)}")

Acuracy: 0.909
Precision YES: 0.909
Precision NO: 0.909
Recall YES: 0.952
Recall NO: 0.833


C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]
C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]


In [227]:
# 3
f1 = f_measure(confusion_matrix1, 1)[0] # Beta = 1
print(f"F1 YES: {round(f1, 3)}")
f1 = f_measure(confusion_matrix1, 1)[1] # Beta = 1
print(f"F1 NO: {round(f1, 3)}")

F1 YES: 0.93
F1 NO: 0.87


C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]
C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]


In [228]:
# 4
# TPR = Recall = TP / (TP + FN)
# FPR = FP / (FP + TN)
positiverates(confusion_matrix1)

print("The difference between recall and TPR is that recall is the ratio of the correctly predicted positive observations to the all observations in actual class and TPR is the ratio of the correctly predicted positive observations to the all observations in predicted class.")

TPR 0.9523809523809523
FPR 0.16666666666666666
The difference between recall and TPR is that recall is the ratio of the correctly predicted positive observations to the all observations in actual class and TPR is the ratio of the correctly predicted positive observations to the all observations in predicted class.


C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TPR = confusion_matrix.iloc[0][0]/confusion_matrix.iloc[0, :].sum()
C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  FPR = confusion_matrix.iloc[1][0]/confusion_matrix.iloc[1, :].sum()


### Question 3:
- In de tabel hieronder vind je de confusion matrix van een binaire classifier.

|            | Predicted YES | Predicted NO | Total |
|------------|---------------|--------------|-------|
| Actual A   | 100           | 0            | 100   |
| Actual B   | 50            | 5            | 55    |
| Total      | 150           | 5            | 155   |

- Beantwoord nu de volgende vragen:
    - Wat zijn de waarden voor TP, TN, FP, FN?
    - Bereken met je zelfgeschreven Python-functies de accuracy, precision en recall, en F1.
    - Is dit een goede classifier?


In [229]:
# 1
# YES = Positive
# NO = Negative
confusion_matrix2 = pd.DataFrame(data=[[100, 0], [50, 5]], columns=['Predicted YES', 'Predicted NO'], index=['Actual A', 'Actual B'])
display(confusion_matrix2)

,Predicted YES,Predicted NO
Actual A,100,0
Actual B,50,5


In [230]:
# 2
accuracy = np.diag(confusion_matrix2).sum()/confusion_matrix2.sum().sum()
print(f"Acuracy: {round(accuracy, 3)}")

precision_yes = precision(confusion_matrix2)[0]
print(f"Precision YES: {round(precision_yes, 3)}")
precision_no = precision(confusion_matrix2)[1]
print(f"Precision NO: {round(precision_no, 3)}")

recall_a = recall(confusion_matrix2)[0]
print(f"Recall A: {round(recall_a, 3)}")
recall_b = recall(confusion_matrix2)[1]
print(f"Recall B: {round(recall_b, 3)}")

f1 = f_measure(confusion_matrix2, 1)[0] # Beta = 1
print(f"F1 YES: {round(f1, 3)}")
f1 = f_measure(confusion_matrix2, 1)[1] # Beta = 1
print(f"F1 NO: {round(f1, 3)}")

Acuracy: 0.677
Precision YES: 0.667
Precision NO: 1.0
Recall A: 1.0
Recall B: 0.091
F1 YES: 0.8
F1 NO: 0.167


C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]
C:\Users\elias\AppData\Local\Temp\ipykernel_24660\2513945939.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TP = confusion_matrix.iloc[i][i]


In [231]:
# 3
if (accuracy < 0.99) | (precision_yes < 0.99) | (recall_a < 0.99):
    print("This is not a good classifier")
else:
    print("This is a good classifier")

This is not a good classifier


### Question 4:
- In de tabel hieronder vind je de confusion matrix van een multiclass classifier.

|            | Asfalt | Beton | Gras | Boom | Gebouw | Total |
|------------|--------|-------|------|------|--------|-------|
| Asfalt     | 2385   | 0     | 0    | 0    | 12     | 2397  |
| Beton      | 4      | 332   | 1    | 0    | 0      | 337   |
| Gras       | 0      | 0     | 908  | 0    | 0      | 908   |
| Boom       | 1      | 0     | 8    | 1084 | 6      | 1099  |
| Gebouw     | 4      | 1     | 0    | 9    | 2053   | 2067  |
| Total      | 2394   | 333   | 917  | 1093 | 2071   | 6808  |

- Beantwoord nu de volgende vragen:
    - Wat zijn de waarden voor TP, FP, FN voor de klasse Gras?
    - Bereken nu de accuracy, precision en recall per klasse met deze functies.
    - Is dit een goede classifier?


In [232]:
# 1
confusion_matrix3 = pd.DataFrame(data=[[2385, 0, 0, 0, 12], [4, 332, 1, 0, 0], [0, 0, 908, 0, 0], [1, 0, 8, 1084, 6], [4, 1, 0, 9, 2053]], columns=['Asfalt', 'Beton', 'Gras', 'Boom', 'Gebouw'], index=['Asfalt', 'Beton', 'Gras', 'Boom', 'Gebouw'])
display(confusion_matrix3)

,Asfalt,Beton,Gras,Boom,Gebouw
Asfalt,2385,0,0,0,12
Beton,4,332,1,0,0
Gras,0,0,908,0,0
Boom,1,0,8,1084,6
Gebouw,4,1,0,9,2053


In [233]:
# 2

In [234]:
# 3

### Question 5:
- Lees de data in het bestand `simpsons_roc1`. 
    - Gebruik de plot_roc functie om de ROC-curve te tekenen.
    - Wat is de beste treshold-waarde?
    - Wat is de AUC? Wat betekent het?
    - Pas de gevonden treshold-waarde (zie b.) toe op de kolom y_score (deze kolom bevat de resultaten van een neuraal netwerk predict_proba-functie -zie later-) om een nieuwe voorspelde kolom te maken.
    - Voer de stappen a. tot en met d. uit met de dataset in het bestand `simpsons_roc2`. Merk je verschillen?


In [235]:
# 1

In [236]:
# 2

In [237]:
# 3

In [238]:
# 4

### Question 6:
- Twee lineaire discriminante analysemodellen werden gebouwd met de dataset-infert -zie bestand infert.csv- De resultaten van de voorspellingen zijn te vinden in de bestanden `Education_roc1.csv` en `Education_roc2.csv`.
    - Laad beide datasets
    - Gebruik de plot_roc functie (zie slides) om beide ROC-curven te tekenen. Gebruik voor beide datasets kolom `6-11yrs` als y-score en transformeer de kolom `y_true` in 1 (voor de waarden `6-11yrs` en een 0 voor de andere waarden).
    - Wat is de beste treshold-waarde voor elk van de ROC-curves?
    - Welk model heeft uw voorkeur?

In [239]:
# 1
education_roc1 = pd.read_csv('../Data/Education_roc1.csv', delimiter=',', decimal='.')
education_roc2 = pd.read_csv('../Data/Education_roc2.csv', delimiter=',', decimal='.')

In [240]:
# 2

In [241]:
# 3

In [242]:
# 4